# Lab 1
## **Text processing**

## Exercise 1:
Benchmark different language-detection algorithm by computing accuracy of each approach.
- FastText
- LangID
- langDetect

Hint: use language code conversion `iso639-lang`

Report
- Accuracy
- Average time per example

In [1]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/langid_dataset.csv

--2022-10-06 21:00:40--  https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/langid_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12990065 (12M) [text/plain]
Saving to: ‘langid_dataset.csv’

langid_dataset.csv  100%[===================>]  12.39M  17.1MB/s    in 0.7s    

2022-10-06 21:00:41 (17.1 MB/s) - ‘langid_dataset.csv’ saved [12990065/12990065]



## My code..

In [15]:
# import
import pandas as pd

# read file
corpus = pd.read_csv('langid_dataset.csv')

In [14]:
corpus.head()

NameError: name 'corpus' is not defined

In [21]:
# fastText (fastlangid)
from fastlangid import LID
langid = LID()
res = langid.predict(corpus.Text[0])
print(res)

et


In [18]:
test = corpus.language[0]

In [24]:
from iso639 import languages
languages.get(alpha2=res).name == test

True

'Estonian'